# Ensemble Analysis

This example compares experimental structural data analyzed using Principal Component Analysis (PCA) with the theoretical data predicted by Anisotropic Network Model (ANM):

## Retrieve dataset

In [ ]:
from prody import *
from pylab import *
%matplotlib inline

In [ ]:
p38 = parsePDB('1p38')

In [ ]:
p38_sequence = p38.getHierView()['A'].getSequence()

In [ ]:
p38_sequence

In order to perform a BLAST search of the PDB, we will need the amino acid sequence of our reference protein. We could get the FASTA format from the PDB, or we could get the sequence from the PDB file itself. A more attractive method (to us) is to get the sequence using ProDy.

In [ ]:
blast_record = blastPDB(p38_sequence)

In [ ]:
pdbids = blast_record.getHits()

Fetching the PDBs that contain the selected sequence can require time. For this reason during our workshop we will use files I previously download and you will find into the subfolder __p38_pdbs__

In [ ]:
import glob

In [ ]:
pdbfiles = glob.glob('your/folder/2.1-File_parsing_Pandas/p38_pdbs/*pdb')

If you want to fetch them instead, you can use the following command

In [ ]:
# pdbfiles = fetchPDB(*pdbids, compressed=False)

## Set reference chain

In [ ]:
ref_structure = parsePDB('1p38')
ref_selection = ref_structure.select('resnum 5 to 31 36 to 114 122 to '
                                     '169 185 to 351 and calpha')

In [ ]:
ref_chain = ref_selection.getHierView().getChain('A')
repr(ref_chain)

## Ensemble Preparation

In [ ]:
ensemble = PDBEnsemble('p38 X-ray')

In [ ]:
ensemble.setAtoms(ref_chain)
ensemble.setCoords(ref_chain)

In [ ]:
for pdbfile in pdbfiles:
    structure = parsePDB(pdbfile, subset='calpha')
    mappings = mapOntoChain(ref_structure, ref_chain)
    atommap = mappings[0][0]
    ensemble.addCoordset(atommap, weights=atommap.getFlags('mapped'), degeneracy=True)

In [ ]:
repr(ensemble)

In [ ]:
len(ensemble) == len(pdbfiles)

In [ ]:
ensemble.iterpose()

## PCA calculation

In [ ]:
pca = PCA('p38 xray')           # Instantiate a PCA instance

pca.buildCovariance(ensemble)   # Build covariance for the ensemble

pca.calcModes()                 # Calculate modes (20 of the by default)

## ANM calculation

In [ ]:
anm = ANM('1p38')             # Instantiate a ANM instance

anm.buildHessian(ref_chain)   # Build Hessian for the reference chain

anm.calcModes()               # Calculate slowest non-trivial 20 modes

## Save your work

In [ ]:
saveModel(pca)
saveModel(anm)
saveEnsemble(ensemble)
writePDB('p38_ref_chain.pdb', ref_chain)

## Fractional Variance

In [ ]:
for mode in pca[:3]:    # Print % variance explained by top PCs
    var = calcFractVariance(mode)*100
    print('{0:s}  % variance = {1:.2f}'.format(mode, var))

## Collectivity of modes

In [ ]:
for mode in pca[:3]:    # Print PCA mode collectivity
    coll = calcCollectivity(mode)
    print('{0:s}  collectivity = {1:.2f}'.format(mode, coll))

In [ ]:
for mode in anm[:3]:    # Print ANM mode collectivity
    coll = calcCollectivity(mode)
    print('{0:s}  collectivity = {1:.2f}'.format(mode, coll))

## PCA - ANM overlap

In [ ]:
printOverlapTable(pca[:3], anm[:3]) # Top 3 PCs vs slowest 3 ANM modes

In [ ]:
showOverlapTable(pca[:6], anm[:6]);
title('PCA - ANM Overlap Table');

In [ ]:
showOverlap(pca[0], anm);

In [ ]:
showCumulOverlap(pca[0], anm);

## Square Fluctuations

In [ ]:
showSqFlucts(pca[:3]);

In [ ]:
showSqFlucts(anm[:3]);

In [ ]:
showScaledSqFlucts(pca[0], anm[2]);
legend();

In [ ]:
showNormedSqFlucts(pca[0], anm[1]);
legend();

## Cross Correlations

In [ ]:
showCrossCorr(anm)

In [ ]:
showCrossCorr(pca)

## Preparation of VMD Output

In [ ]:
writeNMD('p38_anm.nmd',anm,ref_chain)

In [ ]:
writeNMD('p38_pca.nmd',pca,ref_chain)